In [133]:
import math 
import clip
import torch
import numpy as np
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 

In [89]:
model, preprocess = clip.load("ViT-B/16", device=0)

In [90]:
preprocess

Compose(
    Resize(size=224, interpolation=bicubic)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x7f45c50cf940>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [166]:
img = Image.open('TXTARR019023NeighObliq61261S_190124.jpg')

In [167]:
print (img.size)
img = img.resize((int(256* math.ceil(img.size[0]/256)), int(256* math.ceil(img.size[1]/256))))
print (img.size)

(6576, 4384)
(6656, 4608)


In [168]:
labels = ["House",
"Solar Panels",
"Vehicles",
"Swimming Pool",
"Trees and Shrubs",
"Yard Debris",
"Trampoline",
"Water Bodies",
"Power Lines",
"Industry"]

text = clip.tokenize(labels).cuda()
text_features = model.encode_text(text)

In [169]:
with torch.no_grad():
    for i in range(0, img.size[0], 256):
        for j in range(0, img.size[0], 256):
            img_patch = img.crop((i, j, i+256, j+256))
            patch = preprocess(img_patch).unsqueeze(0).cuda()
            image_features = model.encode_image(patch)
            logits_per_image, logits_per_text = model(patch, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()
                        
            draw = ImageDraw.Draw(img)
            # font = ImageFont.truetype("sans-serif.ttf", 16)
            draw.text((i, j), labels[np.argmax(probs)], (255,255,255))
            draw.rectangle((i, j, i+256, j+256), fill=None, outline ="red")

In [170]:
img.save('ev_pred_out_256x256.jpg')